In [102]:
import xml.etree.ElementTree as ET
from sklearn.model_selection import train_test_split

In [105]:
def fileProcessor(F, trainingSize):
    x_train = []   #Training Data
    x_test = []    #Testing Data 
    y_train = []   #Training Classifiers
    y_test = []    #Testing Classifiers
    for file in F:    #F is a vector of file names
        x = [] #data
        y = [] #classifier
        tree = ET.parse(file) #Create XML file tree
        root = tree.getroot() #Get root of tree... In our case root is 'feature_vector_file'
        for child in root:    #Loop through subelements of root
            if child.tag == 'data_set':      #if subelement is 'data_set', perform:
                fileID = child.find('data_set_id').text   #'data_set_id' contains the classifer (i.e the composer name)
                composerName = fileID.split('/')[-2]      #extract composer name from 'data_set_id'
                y.append(composerName)                    #append composer name to classifier vector
                FeatureList = []                          #empty array for List of Features, Features have a name and set of values
                for feature in child.iter('feature'):  #Loop through 'data_set' subelement for features
                    v = []                                #create empty array to contain feature values
                    name = feature.find('name').text      #extract name of feature
                    vals = feature.findall('v')           #extract values of feature
                    for i in vals:                    #loop through extracted values
                        v.append(i.text)                  #append text of values to values array
                    feat = [name, v]                  #join feature name and values
                    FeatureList.append(feat)          #append feature name and values to List of Features
                x.append(FeatureList)           #append List of Features for midi file being analysed to the array of data
        x_trainC, x_testC, y_trainC, y_testC = train_test_split(x, y,test_size=trainingSize) #split data into training and testing data
        #the following is done to ensure that each composer has an equal ratio of music split
        #into training and testing data... although the specific songs by each composer are chosen randomly
        #each composer has an equal ratio of their music split into testing and training data
        x_train = x_train + x_trainC    
        x_test = x_test + x_testC
        y_train = y_train + y_trainC
        y_test = y_test + y_testC
    print(y_train)

In [106]:
v = ['beethovenValues.xml', 'chopinValues.xml']
fileProcessor(v, 0.33)

['Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Beethoven', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin', 'Chopin']
